In [1]:
import re
import numpy as np

from collections import deque
from itertools import islice
DEBUG = False
sample = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2"""

bigger_sample = """R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20"""


In [2]:

def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) --> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = deque(islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)




In [3]:
from sys import maxsize
def print_grid(tailchain):
    array = np.zeros((22,27), dtype=int)
    for i,(y,x) in enumerate(tailchain):
        if i == 0:
            i = -1
        array[y + 6, x + 12] = i
    array[6,12] = maxsize
    for row in np.flip(array, axis=0):
        for col in row:
            print ('s' if col == maxsize else 'H' if col == -1 else col if col else '.', end="")
        print("")



def print_visited(tails):
    array = np.zeros((22,27), dtype=int)
    for (y,x) in tails:
        array[y + 6, x + 12] = 1
    array[6,12] = maxsize
    for row in np.flip(array, axis=0):
        for col in row:
            print ('s' if col == maxsize else '#' if col else '.', end="")
        print("")
    
    


In [9]:



def add_pairs(pair1, pair2):
    return pair1[0] + pair2[0], pair1[1] + pair2[1]
"""
Before: hy=2 hx=1 ty=2 tx=3 yd=0 xd=-2 dist=2
After: hy=2 hx=1 ty=2 tx=3 dist=2

......
......
.H.T..
......
......"""
def adjust_tail(hy, hx, ty, tx):
    yd = hy - ty
    xd = hx - tx
    dist = abs(yd) + abs(xd)
    DEBUG and print(f'Before: {hy=} {hx=} {ty=} {tx=} {yd=} {xd=} {dist=}')
    if abs(yd) > 1 and abs(xd) > 1:
        ty += yd // 2
        tx += xd // 2
    elif abs(yd) > 1 and abs(xd) == 1:
        ty += yd // 2
        tx = hx
    elif abs(xd) > 1 and abs(yd) == 1:
        tx += xd // 2
        ty = hy
    elif abs(xd) == 2:
        tx += xd // 2
    elif abs(yd) == 2:
        ty += yd // 2
    DEBUG and print(f'After: {hy=} {hx=} {ty=} {tx=} {dist=}')
    # DEBUG and print("")
    return ty, tx


dirs = {
    'R': (0, 1),
    'L': (0, -1),
    'U': (1, 0),
    'D': (-1, 0),
    }

dir_letter = {v:k for k,v in dirs.items()}


def get_data(data):
    results = []
    instructions = re.findall(r'([RLUD]) (\d+)', data)
    for dir, amt in instructions:
        results.append((dirs[dir], int(amt)))
    return results

def get_answer(data, tails=1):

    results = {(0,0),}
    tailchain = [(0,0)] * (tails + 1)

    for dir, amt in data:
        DEBUG and print(f"{dir_letter[dir]} {amt}")
        for _ in range(amt):
            for h, t in sliding_window(range(tails + 1), 2):
                DEBUG and print (h,t)
                hy = tailchain[h][0]
                hx = tailchain[h][1]
                ty = tailchain[t][0]
                tx = tailchain[t][1]
                if not h:
                    hy, hx = add_pairs((hy, hx), dir)
                    tailchain[h] = hy,hx
                ty, tx = adjust_tail(hy, hx, ty, tx)
                tailchain[t] = ty,tx
                results.add(tailchain[-1])
        DEBUG and print_grid(tailchain)
        DEBUG and print("")
    DEBUG and print_visited(results)
    return len(results)


DEBUG = False


data = get_data(bigger_sample)
print(get_answer(data, tails=9))
DEBUG = False
data = get_data(open('d9.input').read())

print(get_answer(data))
print(get_answer(data, tails=9))




36
6026
2273
